## Необходимые зависимости

In [1]:
!pip install sentence_transformers
!pip install xlwt
!git clone https://github.com/RusinDmitry/Identifying-implicit-relationships-in-abstract-text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.8 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=f6365a202b2570d877a1116800d6640e40edc693391c5009da052a9f3a9da97b
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.0/100.0 kB 1.3 MB/s eta 0:00:00
Cloning into 'Identifying-implicit-relationships-in-abstract-text'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 43 (delta 17), reused 3 (delta 0), pack-reused 0
Receiving objects: 100% (43/43), 5.39 MiB | 4.20 MiB/s, done.
Resolving deltas: 100% (17/17), done.


## Загрузка данных

In [12]:
import pandas as pd
import joblib
from typing import List
from sentence_transformers import SentenceTransformer, util
import torch
import re

#изначально функция для n-gramm, но используется просто для форматирования
def create_ngrams(lst, n):
    return [' '.join(lst[i:i+n]) for i in range(len(lst)-n+1)]

#удалить все кроме даты
def filter_data(text):
    match = re.findall(r'\d{4}-\d{2}-\d{2}', text)
    if len(match) > 0:
        return match[0]
    else:
        return ''

#удалить все кроме чисел и убрать .00
def filter_price(text):
    match = re.findall(r'([0-9\s.,]+)', text)
    if len(match) > 0:
        price = ' '.join(match)
        price = re.sub('\D', '', price)
        if len(price) > 2:
            price = price[:-2]
        return price
    else:
        return ''

#удалить "производитель:, разработчик" перед названием
def filter_maker(text):
    match = re.findall(r':.+', text)
    if len(match) > 0:
        return match[0][1:]
    else:
        return ''


#вариант для csv файла, в котором объединены applic и meta (есть all_test_df.csv и all_train_df.csv)
file_path = '/content/Identifying-implicit-relationships-in-abstract-text/all_test_df.csv'
data = pd.read_csv(file_path)["Data"].iloc[:]
data = data.apply(lambda x: x.split("\n")).to_list()

#вариант для dat файла (в applic файле мало информации о цене, производителяих итд)
#def read_dat_file(file_path):
#    with open(file_path, 'r') as file:
#        lines = file.readlines()
#
#    data = []
#    current_data = []
#    for line in lines:
#        if line.strip() == "*$*":
#            if current_data:
#                data.append(current_data)
#            current_data = []
#        else:
#            current_data.append(line.strip())
#
#    if current_data:
#        data.append(current_data)
#
#    return data
#file_path = '/content/Identifying-implicit-relationships-in-abstract-text/all_applic.dat'
#data = read_dat_file(file_path)

result = pd.DataFrame(columns = ["Контингент (индификатор)",
                                 "Название проекта (Описание)",
                                 "Дата выхода",
                                 "Цена",
                                 "Производитель, разработчик",
                                 "Покупатель, заказчик",
                                 "Участник торгов"])




filters = {"Дата выхода": filter_data,
           "Производитель, разработчик": filter_maker,
           "Цена": filter_price,
           "Покупатель, заказчик": filter_maker,
           "Участник торгов": filter_maker}



## Загрузка модели вложений предложений

In [3]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(DEVICE)
pretrained_model = SentenceTransformer('distiluse-base-multilingual-cased-v1', device = DEVICE)

cuda


.gitattributes:   0%|          | 0.00/690 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/452 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

## Преобразование предложений из тендоров в вложения

In [4]:
import numpy as np

lens = list(map(len, data))
cut_dot = np.add.accumulate(lens)[:-1]

corpus = sum([sum([create_ngrams(data[i], n) for n in range(1, 2)], []) for i in range(len(data))], [])
embeddings = pretrained_model.encode(corpus, convert_to_tensor=False)
embeddings_list = np.split(embeddings, cut_dot)

## Загрузка модели машинного обучения - парсера

In [5]:
model = joblib.load('/content/Identifying-implicit-relationships-in-abstract-text/model.pkl')

In [13]:
def parse(tender: List[str], embedding):
    inform = {}
    for string, sentence in zip(tender, embedding):
        predict = model.predict(sentence.reshape(1, -1))[0]
        if predict in list(result.columns):
            if predict in filters.keys():
                pre_string = filters[predict](string)
            else:
                pre_string = string

            if predict in inform.keys():
                inform[predict] += ', ' + pre_string
            else:
                inform[predict] = pre_string
    return inform


for data_i, embeddings_i in zip(data, embeddings_list):
    new_row = parse(data_i, embeddings_i)
    new_df = pd.DataFrame(new_row, index=[0])
    result = pd.concat([result, new_df], ignore_index=True)

result.to_csv("table.csv")
result.to_excel("table.xls")
result

<ipython-input-13-189dc12a7b36>:24: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  result.to_excel("table.xls")


,Контингент (индификатор),Название проекта (Описание),Дата выхода,Цена,"Производитель, разработчик","Покупатель, заказчик",Участник торгов
0,NaN,"20220429005), идентифицированные лица:, Нет, Н...",NaN,NaN,NaN,"2022-07-12, P178586@gnpjvc.com.cn","CGN-CGN)., 0755-8447860, (подпись),"
1,NaN,"Нет, Участники торгов: &lt; &lt; Немонгольская...",,NaN,NaN,2022-07-27,"пиковая GIS), , &quot; Чайна инжиниринг лими..."
2,"(Контингент: CGN-202111010012), /.",Объявлены результаты закупок оборудования для ...,NaN,NaN,NaN,2022-08-12,","
3,(Контингент: CGN-20201028007),Конкретная информация о результатах закупок об...,NaN,NaN,NaN,2021-07-29,", Рязанский круг"
4,Электронная почта: liubin518@cgnpc.com.cn,Был определен следующий состав участников прое...,NaN,NaN,NaN,NaN,
...,...,...,...,...,...,...,...
382,NaN,100 МВт ветрогенераторного оборудования (включ...,NaN,NaN,NaN,NaN,","
383,(Контингент: CGN-202304130007),Результаты оценки безопасности гор на Японской...,NaN,NaN,NaN,NaN,", , Лю Цзянь"
384,В рамках этого проекта были отобраны следующие...,Инвентаризаторы: испытательное оборудование дл...,NaN,NaN,,2023-04-12,Закупка экспериментального оборудования по мо...
385,Проект экологизации и восстановления окружающе...,В 20230630003 году были определены следующие у...,NaN,,,"2023-08-24,",","
